Import Statements

In [31]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import numpy as np
import os

def filter_countries(df, column_name, iso_column, iso_values):
    # Create an empty DataFrame with the same columns as 'df'
    filtered_df = pd.DataFrame(columns=df.columns)

    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # Get the ISO value from the current row
        iso = row[iso_column]

        # Check if the ISO value is in the specified list of ISO values
        if iso in iso_values:
            # Append the row to the filtered DataFrame
            filtered_df = pd.concat([filtered_df, row.to_frame().T])

    # Reset the index of the filtered DataFrame
    filtered_df.reset_index(drop=True, inplace=True)

    # Return the filtered DataFrame
    return filtered_df

def filter_countriesForSuicide(df, column_name, country_column, country_values):
    # Create an empty DataFrame with the same columns as 'df'
    filtered_df = pd.DataFrame(columns=df.columns)

    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # Get the country name from the current row
        country = row[country_column]

        # Check if the country name is in the specified list of country values
        if country in country_values:
            # Append the row to the filtered DataFrame
            filtered_df = pd.concat([filtered_df, row.to_frame().T])

    # Reset the index of the filtered DataFrame
    filtered_df.reset_index(drop=True, inplace=True)

    # Return the filtered DataFrame
    return filtered_df


KaggleApi Settings

In [41]:
api = KaggleApi()
api.authenticate()

Import Datasets with Kaggle.API

In [42]:
api.dataset_download_file('thedevastator/global-fossil-co2-emissions-by-country-2002-2022',
                          file_name='GCB2022v27_MtCO2_flat.csv')
with zipfile.ZipFile('GCB2022v27_MtCO2_flat.csv.zip', 'r') as zipref:
    zipref.extractall()

In [43]:
api.dataset_download_file('russellyates88/suicide-rates-overview-1985-to-2016',
                          file_name='master.csv')
with zipfile.ZipFile('master.csv.zip', 'r') as zipref:
    zipref.extractall()

In [44]:
api.dataset_download_file('yapwh1208/countries-gdp-2012-to-2021', file_name='GDP.csv')

False

In [45]:
api.dataset_download_file('folaraz/world-countries-and-continents-details',
                          file_name='Countries Longitude and Latitude.csv')

False

In [59]:
#list with european countries with their iso codes
df_countries_EUROPE = pd.read_csv(r'Country_ISOCode.csv', delimiter=',')
df_countries_EUROPE


,Country,ISO Code
0,Albania,ALB
1,Andorra,AND
2,Austria,AUT
3,Belarus,BLR
4,Belgium,BEL
5,Bosnia and Herzegovina,BIH
6,Bulgaria,BGR
7,Croatia,HRV
8,Czech Republic,CZE
9,Denmark,DNK


Read and Filter DataFiles

In [76]:
df_emissions = pd.read_csv("GCB2022v27_MtCO2_flat.csv", sep=",")

new_df_emissions = df_emissions[(df_emissions['Year'] >= 2006) & (df_emissions['Year'] <= 2016)]
new_df_emissions.rename(columns={'Year':'year', 'Country':'country', 'Total':'Total amount of CO2'}, inplace=True)
new_df_emissions = new_df_emissions.drop(['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita'], axis=1)
new_df_emissions

european_df_newDfEmission = filter_countries(new_df_emissions, 'column_name', 'ISO 3166-1 alpha-3', df_countries_EUROPE['ISO Code'].values)
print(european_df_newDfEmission)


C:\Users\Admin\AppData\Local\Temp\ipykernel_27420\1704984300.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_emissions.rename(columns={'Year':'year', 'Country':'country', 'Total':'Total amount of CO2'}, inplace=True)


            country ISO 3166-1 alpha-3  year Total amount of CO2
0           Albania                ALB  2006            3.910715
1           Albania                ALB  2007             3.94897
2           Albania                ALB  2008            4.397043
3           Albania                ALB  2009            4.406552
4           Albania                ALB  2010            4.783865
..              ...                ...   ...                 ...
468  United Kingdom                GBR  2012          487.476776
469  United Kingdom                GBR  2013          477.611268
470  United Kingdom                GBR  2014          438.807081
471  United Kingdom                GBR  2015          422.460794
472  United Kingdom                GBR  2016          399.430131

[473 rows x 4 columns]


In [77]:
df_suicide = pd.read_csv("master.csv", sep=",")

new_df_suicide = df_suicide[(df_suicide['year'] >= 2006) & (df_suicide['year'] <= 2016)]
# Group the dataframe by 'country' and 'year' and calculate the sum of 'suicides_no'
new_df_suicide = new_df_suicide.groupby(['country', 'year'])['suicides_no'].sum(). reset_index()
#new_df_suicide = new_df_suicide.drop(['HDI for year', 'gdp_per_capita ($)'], axis=1) -not necessary

new_df_suicide

european_df_newDfSuicide = filter_countries(new_df_suicide, 'column_name', 'country', df_countries_EUROPE['Country'].values)
print(european_df_newDfSuicide)

#compare both dataframes to see if merge possible on country
column1 = european_df_newDfSuicide['country']
column2 = european_df_newDfEmission['country']

# Find the values in column1 that are not in column2
differences = column1[~column1.isin(column2)]

# Convert the differences to a list
differences_list = differences.tolist()

# Print the list of differences
print("Values that are not the same:")
for value in differences_list:
    print(value)


            country  year suicides_no
0           Albania  2006           0
1           Albania  2007         124
2           Albania  2008         160
3           Albania  2009           0
4           Albania  2010          96
..              ...   ...         ...
350  United Kingdom  2011        4393
351  United Kingdom  2012        4444
352  United Kingdom  2013        4824
353  United Kingdom  2014        4788
354  United Kingdom  2015        4910

[355 rows x 3 columns]
Values that are not the same:


Conditional statements, loop control statements and loops

In [48]:
df_gdp = pd.read_csv("GDP.csv", sep=",")

new_df_gdp = df_gdp[['Country Name', 'Country Code', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']]
new_df_gdp = new_df_gdp.melt(id_vars=["Country Name", "Country Code"], var_name="year")
new_df_gdp['year'] = new_df_gdp['year'].astype('int64')
new_df_gdp.rename(columns={'Country Name':'country', 'Country Code': 'ISO 3166-1 alpha-3', 'value':'GDP_Value'}, inplace=True)



In [49]:
df_allCountries = pd.read_csv("Countries%20Longitude%20and%20Latitude.csv", sep=",")
df_allCountries.rename(columns={'name':'country'}, inplace=True)


In [79]:

#dataframes should be limited to europe
european_df_newDfGdp = filter_countries(new_df_gdp, 'column_name', 'ISO 3166-1 alpha-3', df_countries_EUROPE['ISO Code'].values)
print(european_df_newDfGdp)

#compare both dataframes to see if merge possible on ISO
column3 = european_df_newDfGdp['ISO 3166-1 alpha-3']
column4 = european_df_newDfEmission['ISO 3166-1 alpha-3']

# Find the values in column1 that are not in column2
differences1 = column1[~column1.isin(column2)]

# Convert the differences to a list
differences_list1 = differences1.tolist()

# Print the list of differences
print("Values that are not the same:")
for value in differences_list1:
    print(value)





             country ISO 3166-1 alpha-3  year    GDP_Value
0            Albania                ALB  2006  2972.742924
1            Andorra                AND  2006  43086.49983
2            Austria                AUT  2006  40669.32696
3            Belgium                BEL  2006   38705.1068
4           Bulgaria                BGR  2006  4523.050833
..               ...                ...   ...          ...
490  Slovak Republic                SVK  2016   16563.4405
491         Slovenia                SVN  2016  21678.35947
492           Sweden                SWE  2016  51965.15715
493          Turkiye                TUR  2016  10734.37997
494          Ukraine                UKR  2016  2187.730469

[495 rows x 4 columns]
Values that are not the same:


kombinieren


In [91]:
merged_df_emissions_suicide = pd.merge(new_df_emissions, new_df_suicide, on=['country', 'year'])
merged_df_emissions_suicide = pd.merge(merged_df_emissions_suicide, df_allCountries, on=['country'])
merged_df_emissions_suicide = merged_df_emissions_suicide.drop('Unnamed: 0', axis=1)


In [80]:
#merge on ISO and Year
merged_df_emissions_gdp = pd.merge(european_df_newDfEmission, european_df_newDfGdp, on=['ISO 3166-1 alpha-3', 'year'])
merged_df_emissions_gdp = merged_df_emissions_gdp.drop(['country_y'], axis=1)
##merged_df_emissions_gdp = pd.merge(merged_df_emissions_gdp, df_allCountries, on=['country'])
merged_df_emissions_gdp.rename(columns={'country_x':'country'}, inplace=True)
#change the order of the dateframe - maybe needed for linear regression
# Create a list of column names in the desired order
column_order = ['country', 'ISO 3166-1 alpha-3', 'year', 'GDP_Value', 'Total amount of CO2']

# Reindex the DataFrame with the specified column order
merged_df_emissions_gdp = merged_df_emissions_gdp.reindex(columns=column_order)
merged_df_emissions_gdp
##merged_df_emissions_gdp = merged_df_emissions_gdp.drop('Unnamed: 0', axis=1)


,country,ISO 3166-1 alpha-3,year,GDP_Value,Total amount of CO2
0,Albania,ALB,2006,2972.742924,3.910715
1,Albania,ALB,2007,3595.038057,3.94897
2,Albania,ALB,2008,4370.539925,4.397043
3,Albania,ALB,2009,4114.134899,4.406552
4,Albania,ALB,2010,4094.348386,4.783865
...,...,...,...,...,...
468,United Kingdom,GBR,2012,42485.58607,487.476776
469,United Kingdom,GBR,2013,43449.09172,477.611268
470,United Kingdom,GBR,2014,47447.58893,438.807081
471,United Kingdom,GBR,2015,45071.07432,422.460794


In [93]:
merged_df_suicide_gdp = pd.merge(new_df_suicide, new_df_gdp, on=['country', 'year'])
merged_df_suicide_gdp = pd.merge(merged_df_suicide_gdp, df_allCountries, on=['country'])
merged_df_suicide_gdp = merged_df_suicide_gdp.drop('Unnamed: 0', axis=1)

Use of Python built-in data structures (lists, dictionaries, sets, tuples)

In [94]:
merged_df_emissions_suicide = merged_df_emissions_suicide.fillna(0)
merged_df_emissions_gdp = merged_df_emissions_gdp.fillna(0)
merged_df_suicide_gdp = merged_df_suicide_gdp.fillna(0)
#df_countries = df_countries.fillna(0)
df_allCountries = df_allCountries.fillna(0)

Use of tables, vizualizations/graphics for data exploration

Integration of a statistical analyses (correlation analysis, statical tests)

In [95]:
merged_df_emissions_gdp.to_csv(r'final_csv/merged_df_emissions_gdp.csv', index=False, header=True)
merged_df_suicide_gdp.to_csv(r'final_csv/merged_df_suicide_gdp.csv', index=False, header=True)
merged_df_emissions_suicide.to_csv(r'final_csv/merged_df_emissions_suicide.csv', index=False, header=True)

Remove all non-used files

In [96]:
os.remove("master.csv")
os.remove("master.csv.zip")
os.remove("GDP.csv")
os.remove("GCB2022v27_MtCO2_flat.csv.zip")
os.remove("GCB2022v27_MtCO2_flat.csv")